In [ ]:
import pandas as pd
import re
import os
from google.cloud import storage

# Nếu bạn có service account key file, uncomment dòng dưới:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/service-account-key.json'

df = pd.read_csv("batdongsan.csv")


In [2]:
df.head(5)


,Diện tích sử dụng,Mã BĐS,price,Pháp lý,Ngày đăng,Nhà tắm,Phòng ngủ,address,Diện tích đất
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [3]:
df = df.rename(columns={
    'Diện tích sử dụng' : 'dien_tich_su_dung_m2',
    'Mã BĐS' : 'ma_bds',
    'price' : 'gia_ty',
    'Pháp lý' : 'phap_ly',
    'Ngày đăng' : 'ngay_dang',
    'Nhà tắm'  : 'nha_tam',
    'Phòng ngủ': 'phong_ngu',
    'address': 'dia_chi',
    'Diện tích đất': 'dien_tich_dat_m2'
})

In [4]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2
0,100 m2,22712100,5 tỷ 800 triệu,Sổ đỏ,15/10/2025,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN
1,80 m2,22712094,4 tỷ 350 triệu,Sổ đỏ,15/10/2025,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
2,90 m2,22711721,4 tỷ 580 triệu,Sổ đỏ,15/10/2025,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN
3,120 m2,22711722,4 tỷ 700 triệu,Sổ đỏ,15/10/2025,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN
4,100 m2,22711723,5 tỷ 400 triệu,Sổ đỏ,15/10/2025,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN


In [5]:
# dien  tich su dung
df['dien_tich_su_dung_m2'] = df['dien_tich_su_dung_m2'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)



In [6]:
df = df.drop_duplicates(subset=['ma_bds']).reset_index(drop=True)
df['ma_bds'] = df['ma_bds'].astype(str)
df['nha_tam'] = pd.to_numeric(df['nha_tam'], errors='coerce').astype('Int64')
df['phong_ngu'] = pd.to_numeric(df['phong_ngu'], errors='coerce').astype('Int64')

In [7]:
def chuan_hoa_gia(x):
    x = str(x).lower()
    ty = 0
    trieu = 0

    # Lấy phần 'tỷ'
    match_ty = re.search(r'(\d+(?:[\.,]\d+)?)\s*t[ỷi]', x)
    if match_ty:
        ty = float(match_ty.group(1).replace(',', '.'))

    # Lấy phần 'triệu'
    match_trieu = re.search(r'(\d+(?:[\.,]\d+)?)\s*triệu', x)
    if match_trieu:
        trieu = float(match_trieu.group(1).replace(',', '.'))

    # Đổi về tỷ đồng
    return round(ty + trieu/1000, 3)

def tach_dia_chi(dia_chi):
    dia_chi = str(dia_chi)
    
    # Tên đường (trước dấu phẩy đầu tiên)
    ten_duong = dia_chi.split(",")[0].strip()

    # Tên phường/xã
    phuong = re.search(r'Phường\s+([^,]+)', dia_chi)
    phuong = phuong.group(1).strip() if phuong else None

    # Tên quận/huyện
    quan = re.search(r'Quận\s+([^,]+)', dia_chi)
    quan = quan.group(1).strip() if quan else None

    # Thành phố / Tỉnh
    tp = re.search(r'(Thành phố|TP\.?|Tỉnh)\s+([^,]+)', dia_chi)
    tp = tp.group(2).strip() if tp else "Hà Nội"

    # Gộp tên đường + phường
    phuong_xa = f"{ten_duong} - Phường {phuong}" if phuong else ten_duong

    return pd.Series([phuong_xa, quan, tp])


# Hàm làm sạch diện tích
def clean_dien_tich(value):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    
    # Tìm số đầu tiên trong chuỗi (có thể có thập phân)
    match = re.search(r'(\d+(\.\d+)?)', value)
    if match:
        return float(match.group(1))
    else:
        return None



df['dien_tich_dat_m2'] = df['dien_tich_dat_m2'].apply(clean_dien_tich)
df['gia(ty)'] = df['gia_ty'].apply(chuan_hoa_gia)
df[['phuong_xa', 'quan_huyen', 'thanh_pho']] = df['dia_chi'].apply(tach_dia_chi)




In [8]:
for c in ['phap_ly','phuong_xa','quan_huyen','thanh_pho']:
    df[c] = df[c].astype(str).str.strip()

In [9]:
df['ngay_dang'] = pd.to_datetime(df['ngay_dang'], format='%d/%m/%Y', errors='coerce')

In [10]:
df.head(5)

,dien_tich_su_dung_m2,ma_bds,gia_ty,phap_ly,ngay_dang,nha_tam,phong_ngu,dia_chi,dien_tich_dat_m2,gia(ty),phuong_xa,quan_huyen,thanh_pho
0,100.0,22712100,5 tỷ 800 triệu,Sổ đỏ,2025-10-15,1,2,"Đại La, Phường Đồng Nhân, Quận Hai Bà Trưng, H...",NaN,5.80,Đại La - Phường Đồng Nhân,Hai Bà Trưng,Hà Nội
1,80.0,22712094,4 tỷ 350 triệu,Sổ đỏ,2025-10-15,2,2,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.35,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
2,90.0,22711721,4 tỷ 580 triệu,Sổ đỏ,2025-10-15,2,3,"Nhân Hòa, Phường Nhân Chính, Quận Thanh Xuân, ...",NaN,4.58,Nhân Hòa - Phường Nhân Chính,Thanh Xuân,Hà Nội
3,120.0,22711722,4 tỷ 700 triệu,Sổ đỏ,2025-10-15,2,3,"Thanh Nhàn, Phường Thanh Nhàn, Quận Hai Bà Trư...",NaN,4.70,Thanh Nhàn - Phường Thanh Nhàn,Hai Bà Trưng,Hà Nội
4,100.0,22711723,5 tỷ 400 triệu,Sổ đỏ,2025-10-15,2,3,"Yên Lạc, Phường Vĩnh Tuy, Quận Hai Bà Trưng, H...",NaN,5.40,Yên Lạc - Phường Vĩnh Tuy,Hai Bà Trưng,Hà Nội


In [38]:

df_tmp = df[['ma_bds','dien_tich_su_dung_m2','gia(ty)','phap_ly','ngay_dang','phuong_xa','quan_huyen','thanh_pho','nha_tam','phong_ngu','dien_tich_dat_m2']]


df_tmp = df_tmp[(df_tmp['gia(ty)'] != 0.0) & (df_tmp['gia(ty)'].notna())]


In [13]:
# KIỂM TRA BUSINESS LOGIC VALIDATION
print("=== BUSINESS LOGIC VALIDATION ===")

# 1. Kiểm tra giá hợp lý
print("1. GIÁ HỢP LÝ:")
print(f"   - Giá min: {df_tmp['gia(ty)'].min():.3f} tỷ")
print(f"   - Giá max: {df_tmp['gia(ty)'].max():.3f} tỷ")
print(f"   - Giá trung bình: {df_tmp['gia(ty)'].mean():.3f} tỷ")

# 2. Kiểm tra diện tích hợp lý
print("\n2. DIỆN TÍCH HỢP LÝ:")
print(f"   - Diện tích min: {df_tmp['dien_tich_su_dung_m2'].min():.1f} m²")
print(f"   - Diện tích max: {df_tmp['dien_tich_su_dung_m2'].max():.1f} m²")
print(f"   - Diện tích trung bình: {df_tmp['dien_tich_su_dung_m2'].mean():.1f} m²")

# 3. Tính giá/m²
df_tmp['gia_per_m2'] = (df_tmp['gia(ty)'] * 1000) / df_tmp['dien_tich_su_dung_m2']
print(f"\n3. GIÁ/M²:")
print(f"   - Giá/m² min: {df_tmp['gia_per_m2'].min():.0f} triệu VND/m²")
print(f"   - Giá/m² max: {df_tmp['gia_per_m2'].max():.0f} triệu VND/m²")
print(f"   - Giá/m² trung bình: {df_tmp['gia_per_m2'].mean():.0f} triệu VND/m²")


=== BUSINESS LOGIC VALIDATION ===
1. GIÁ HỢP LÝ:
   - Giá min: 0.000 tỷ
   - Giá max: 86900.000 tỷ
   - Giá trung bình: 51.110 tỷ

2. DIỆN TÍCH HỢP LÝ:
   - Diện tích min: 1.0 m²
   - Diện tích max: 6300.0 m²
   - Diện tích trung bình: 126.6 m²

3. GIÁ/M²:
   - Giá/m² min: 0 triệu VND/m²
   - Giá/m² max: 790000 triệu VND/m²
   - Giá/m² trung bình: 713 triệu VND/m²


C:\Users\Admin\AppData\Local\Temp\ipykernel_18096\3357212403.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['gia_per_m2'] = (df_tmp['gia(ty)'] * 1000) / df_tmp['dien_tich_su_dung_m2']


In [40]:
df_tmp.to_csv("batdongsan_clean.csv", index=False, encoding="utf-8-sig")


In [41]:
df_tmp[df_tmp['gia(ty)'] == 0.0]

,ma_bds,dien_tich_su_dung_m2,gia(ty),phap_ly,ngay_dang,phuong_xa,quan_huyen,thanh_pho,nha_tam,phong_ngu,dien_tich_dat_m2
